### Cross-Database Data Integration and Analysis Project - SPOTIFY 

In [519]:
#import necessary library packages:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import credentials
import os
import time 
import datetime
import requests
import numpy as np
import pandas as pd
import mysql.connector 
import pymysql
from sqlalchemy import create_engine
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from neo4j import GraphDatabase
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql.functions import col
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [520]:
#set up credentials to access Spotify APi data:
client_id = credentials.client_id
client_secret = credentials.client_secret


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

This code retrieves 10000 tracks of 2010 from Spotify. The fetched data is loaded into to databases: MySql database, MongoDB  and Neo4j graph database, also saved in CSV files.

#### Data fetching

In [431]:
#retrieving Spotify track data from 2010:
artist_id = []
artist = []
track_id = []
track = []
track_popularity = []
album = [] 
release_date = []

for i in range(0,1000,50):
    track_results = sp.search(q="year:2010", type="track", limit=50,offset=i)
    for i, t in enumerate(track_results["tracks"]["items"]):
        artist_id.append(t["artists"][0]["id"])
        artist.append(t["artists"][0]["name"])
        track_id.append(t["id"])
        track.append(t["name"])
        track_popularity.append(t["popularity"])
        album.append(t["album"]["name"])
        release_date.append(t["album"]["release_date"])

In [432]:
#creating pandas DataFrame:
data_df = pd.DataFrame({"artist_id":artist_id,"artist":artist,"track_id":track_id,"track":track,"track_popularity":track_popularity,"album":album,"release_date":release_date} )
print(data_df.shape)

(1000, 7)


In [433]:
data_df

,artist_id,artist,track_id,track,track_popularity,album,release_date
0,360IAlyVv4PCEVjgyMZrxK,Miguel,0JXXNGljqupsJaZsgSbMZV,Sure Thing,89,All I Want Is You,2010-11-26
1,4LLpKhyESsyAXpc4laK94U,Mac Miller,7FAFkQQZFeNwOFzTrSDFIh,The Spins,84,K.I.D.S. (Deluxe),2010-08-13
2,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,86,Doo-Wops & Hooligans,2010-10-05
3,5jTtGLk1mGFMY5lQOvJYUj,bôa,42qNWdLKCI41S4uzfamhFM,Duvet,85,Twilight,2010-04-20
4,6f4XkbvYlXMH0QgVRzW0sM,Waka Flocka Flame,03tqyYWC9Um2ZqU0ZN849H,No Hands (feat. Roscoe Dash & Wale),79,Flockaveli,2010-10-01
...,...,...,...,...,...,...,...
995,5PokPZn11xzZXyXSfnvIM3,Lifehouse,5dePpbamNezj1qhHKJPViS,All In,49,Smoke & Mirrors (Deluxe),2010
996,535ap2f16rTOKTMPTkvbGB,Los Buitres De Culiacan Sinaloa,7mxukykLaEKZp0VtCSOMli,Pinche Gente Alboretera,55,El Movimiento Alterado Vol. 3,2010
997,0pqGj6vO9YHsXuZmaJaP2Y,Jan & Dean,1Oewmmr12xuhgzvUJ9Q0cf,Surf City,52,Surf City And Other Swingin' Cities,2010-01-01
998,3a9qv6NLHnsVxJUtKOMHvD,The Glitch Mob,5VWVVSABdgccboSdrKGiZH,Fortune Days,53,Drink the Sea,2010-06-08


In [434]:
#add additional data to DataFrame:
artist_popularity = []
artist_genres = []
artist_followers = []

for a_id in data_df.artist_id:
    artist = sp.artist(a_id)
    artist_popularity.append(artist["popularity"])
    artist_genres.append(artist["genres"])
    artist_followers.append(artist["followers"]["total"])

In [435]:
data_df = data_df.assign(artist_popularity=artist_popularity, artist_genres=artist_genres, artist_followers=artist_followers)
data_df

,artist_id,artist,track_id,track,track_popularity,album,release_date,artist_popularity,artist_genres,artist_followers
0,360IAlyVv4PCEVjgyMZrxK,Miguel,0JXXNGljqupsJaZsgSbMZV,Sure Thing,89,All I Want Is You,2010-11-26,77,"[r&b, urban contemporary]",4788498
1,4LLpKhyESsyAXpc4laK94U,Mac Miller,7FAFkQQZFeNwOFzTrSDFIh,The Spins,84,K.I.D.S. (Deluxe),2010-08-13,82,"[hip hop, pittsburgh rap, pop rap, rap]",9497713
2,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,86,Doo-Wops & Hooligans,2010-10-05,87,"[dance pop, pop]",50939168
3,5jTtGLk1mGFMY5lQOvJYUj,bôa,42qNWdLKCI41S4uzfamhFM,Duvet,85,Twilight,2010-04-20,66,[],438554
4,6f4XkbvYlXMH0QgVRzW0sM,Waka Flocka Flame,03tqyYWC9Um2ZqU0ZN849H,No Hands (feat. Roscoe Dash & Wale),79,Flockaveli,2010-10-01,64,"[atl hip hop, dirty south rap, pop rap, rap, s...",2714590
...,...,...,...,...,...,...,...,...,...,...
995,5PokPZn11xzZXyXSfnvIM3,Lifehouse,5dePpbamNezj1qhHKJPViS,All In,49,Smoke & Mirrors (Deluxe),2010,62,"[neo mellow, pop rock, post-grunge]",1972276
996,535ap2f16rTOKTMPTkvbGB,Los Buitres De Culiacan Sinaloa,7mxukykLaEKZp0VtCSOMli,Pinche Gente Alboretera,55,El Movimiento Alterado Vol. 3,2010,59,"[corrido, musica mexicana, norteno, sierreno]",976842
997,0pqGj6vO9YHsXuZmaJaP2Y,Jan & Dean,1Oewmmr12xuhgzvUJ9Q0cf,Surf City,52,Surf City And Other Swingin' Cities,2010-01-01,38,"[bubblegum pop, classic garage rock, doo-wop, ...",106177
998,3a9qv6NLHnsVxJUtKOMHvD,The Glitch Mob,5VWVVSABdgccboSdrKGiZH,Fortune Days,53,Drink the Sea,2010-06-08,59,"[glitch, glitch hop, indietronica]",555130


In [ ]:
#sort by tarck popularity@
track_pop_df = 

In [562]:
#get track audio features for each track year 2010:https:
acousticness = []
danceability = []
energy = []
instrumentalness = []
loudness = []
tempo = []

track_count = 0

for track in data_df.track_id:
    features = sp.audio_features(track)
    acousticness.append(features[0]["acousticness"])
    danceability.append(features[0]["danceability"])
    energy.append(features[0]["energy"])
    instrumentalness.append(features[0]["instrumentalness"])      
    loudness.append(features[0]["loudness"])      
    tempo.append(features[0]["tempo"])

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=0JXXNGljqupsJaZsgSbMZV:
 Max Retries, reason: too many 429 error responses

In [522]:
data_df = data_df.assign(acousticness=acousticness,danceability=danceability,energy=energy,instrumentalness=instrumentalness,loudness=loudness,tempo=tempo)
data_df

ValueError: Length of values (1) does not match length of index (1000)

In [487]:
#creating pandas DataFrame for MySql database and saving to csv file:
track_df = data_df[["track","artist","album","track_popularity", "acousticness","danceability","energy","instrumentalness","loudness","tempo"]]
track_df

KeyError: "['acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'tempo'] not in index"

In [440]:
track_df.to_csv("spotify_mysql.csv")

In [236]:
#creating pandas DataFrame for MongoDB and saving to csv file::
artist_df = data_df[["artist","artist_genres","artist_popularity","artist_followers"]]
artist_df

,artist,artist_genres,artist_popularity,artist_followers
0,Miguel,[r&b],77,4788498
1,Mac Miller,"[hip hop, pittsburgh rap, pop rap, rap]",82,9497713
2,Bruno Mars,"[dance pop, pop]",87,50939168
3,bôa,[],66,438554
4,Waka Flocka Flame,"[atl hip hop, dirty south rap, pop rap, rap, s...",64,2714590
...,...,...,...,...
995,Lifehouse,"[neo mellow, pop rock, post-grunge]",62,1972276
996,Los Buitres De Culiacan Sinaloa,"[corrido, musica mexicana, norteno, sierreno]",59,976842
997,Jan & Dean,"[bubblegum pop, classic garage rock, doo-wop, ...",38,106177
998,The Glitch Mob,"[glitch, glitch hop, indietronica]",59,555130


In [237]:
artist_df.to_csv("spotify_mongodb.csv")

In [238]:
#creating pandas DataFrame for Neo4j and saving to csv file:
artist_album_df = data_df[["artist","track","album"]]
artist_album_df

,artist,track,album
0,Miguel,Sure Thing,All I Want Is You
1,Mac Miller,The Spins,K.I.D.S. (Deluxe)
2,Bruno Mars,Just the Way You Are,Doo-Wops & Hooligans
3,bôa,Duvet,Twilight
4,Waka Flocka Flame,No Hands (feat. Roscoe Dash & Wale),Flockaveli
...,...,...,...
995,Lifehouse,All In,Smoke & Mirrors (Deluxe)
996,Los Buitres De Culiacan Sinaloa,Pinche Gente Alboretera,El Movimiento Alterado Vol. 3
997,Jan & Dean,Surf City,Surf City And Other Swingin' Cities
998,The Glitch Mob,Fortune Days,Drink the Sea


In [239]:
artist_album_df.to_csv("spotify_neo4j.csv")

#### Load data into databases:

In [19]:
#to Mysql:
connection = pymysql.connect(host="localhost", user="root",  password="", database="spotify_db")
cursor = connection.cursor()

create_table_query = """
    CREATE TABLE IF NOT EXISTS tracks (
        track_id INT AUTO_INCREMENT PRIMARY KEY,
        track VARCHAR(60),
        artist VARCHAR(60),
        album VARCHAR(60),
        track_popularity INT,
        acousticness FLOAT,
        danceability FLOAT,
        energy FLOAT,
        instrumentalness FLOAT,
        loudness FLOAT,
        tempo FLOAT
        )
    """

cursor.execute(create_table_query)

In [ ]:
# Insert whole DataFrame into MySQL
track_df.to_sql("tracks", con = engine, if_exists = "append", chunksize = 1000, index=False)

In [ ]:
connection.commit()
connection.close()

In [22]:
#to MongoDB:
data = pd.read_csv("spotify_mongodb.csv")
uri = "uri_mongo"
client = MongoClient(uri, server_api=ServerApi('1'))

In [23]:
db = client["SpotifyDB"]
collection = db["Spotifydata"]

In [24]:
data.reset_index(inplace=True)
data_dict = data.to_dict("records")

In [26]:
collection.insert_many(data_dict)

In [ ]:
client.close()

In [427]:
#to Neo4j: --->to finish up
import py2neo
from py2neo import Graph,Node,Relationship

uri = "neo4j+ssc://14c452fc.databases.neo4j.io"
username = "username"
password = "password"

driver = GraphDatabase.driver(uri,auth=(username,password))

In [423]:
cql_create = """CREATE (artist:Artist {
                   "name: $name",
                   "track: $track",
                   "album: $album"}
                   
            )"""

#### Extract data from databases:

In [ ]:
#MySQL:
import mysql.connector 

connection = mysql.connector.connect(host="localhost", user="root",  password="", database="spotify_db")

sql_select_query = "select * from tracks"
cursor = connection.cursor()
cursor.execute(sql_select_query)

records = cursor.fetchall()
records

In [ ]:
#MongoDB:
uri = "uri_mongo"
client = pymongo.MongoClient(uri, server_api=ServerApi('1'))
db = client["SpotifyDB"]
collection = db["Spotifydata"]

x = collection.find()
 
for data in x:
    print(data)

In [ ]:
#Neo4j: ---> not finished yet


#### Data transformation & Integration:

In [488]:
#clean and transform the extracted data using PySpark:
spark = SparkSession.builder.appName("Dataframe").getOrCreate()

In [489]:
mysql_spark = spark.read.csv("spotify_mysql.csv", header=True, inferSchema=True)
mongodb_spark = spark.read.csv("spotify_mongodb.csv", header=True, inferSchema=True)
neo4j_spark = spark.read.csv("spotify_neo4j.csv", header=True, inferSchema=True)

df = [mysql_spark, mongodb_spark, neo4j_spark]

In [490]:
mysql_spark.columns

['_c0', 'track', 'artist', 'album', 'track_popularity']

In [491]:
mongodb_spark.columns,

(['_c0', 'artist', 'artist_genres', 'artist_popularity', 'artist_followers'],)

In [492]:
neo4j_spark.columns

['_c0', 'artist', 'track', 'album']

In [493]:
#join 3 pyspark DataFrames by chaining
merged = mysql_spark.join(mongodb_spark,["_c0"]) \
     .join(neo4j_spark,mysql_spark["_c0"] == neo4j_spark["_c0"])

In [494]:
merged_df = mysql_spark.join(mongodb_spark,["_c0","artist"])

In [495]:
merged_df.show(truncate=True)

+---+--------------------+--------------------+--------------------+----------------+--------------------+-----------------+----------------+
|_c0|              artist|               track|               album|track_popularity|       artist_genres|artist_popularity|artist_followers|
+---+--------------------+--------------------+--------------------+----------------+--------------------+-----------------+----------------+
|  0|              Miguel|          Sure Thing|   All I Want Is You|              89|             ['r&b']|               77|         4788498|
|  1|          Mac Miller|           The Spins|   K.I.D.S. (Deluxe)|              84|['hip hop', 'pitt...|               82|         9497713|
|  2|          Bruno Mars|Just the Way You Are|Doo-Wops & Hooligans|              86|['dance pop', 'pop']|               87|        50939168|
|  5|              Miguel|Girl With The Tat...|   All I Want Is You|              82|             ['r&b']|               77|         4788498|
|  6| 

In [496]:
#verify data types:
merged_df.dtypes

[('_c0', 'int'),
 ('artist', 'string'),
 ('track', 'string'),
 ('album', 'string'),
 ('track_popularity', 'int'),
 ('artist_genres', 'string'),
 ('artist_popularity', 'string'),
 ('artist_followers', 'string')]

In [497]:
#find count of Null, None, NaN of all data columns:
from pyspark.sql.functions import col,isnan, when, count
merged_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in merged_df.columns]).show()

+---+------+-----+-----+----------------+-------------+-----------------+----------------+
|_c0|artist|track|album|track_popularity|artist_genres|artist_popularity|artist_followers|
+---+------+-----+-----+----------------+-------------+-----------------+----------------+
|  0|     0|    0|    0|               0|            0|                0|               0|
+---+------+-----+-----+----------------+-------------+-----------------+----------------+



In [498]:
#replace column name:
renamed_df = merged_df.withColumnRenamed("_c0","No").printSchema()

root
 |-- No: integer (nullable = true)
 |-- artist: string (nullable = true)
 |-- track: string (nullable = true)
 |-- album: string (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- artist_genres: string (nullable = true)
 |-- artist_popularity: string (nullable = true)
 |-- artist_followers: string (nullable = true)



In [499]:
#check duplicate values in DataFrame:
merged_df = merged_df.distinct()
print("Distinct count: "+str(merged_df.count()))
merged_df.show(truncate=True)

Distinct count: 258
+---+--------------------+--------------------+--------------------+----------------+--------------------+-----------------+----------------+
|_c0|              artist|               track|               album|track_popularity|       artist_genres|artist_popularity|artist_followers|
+---+--------------------+--------------------+--------------------+----------------+--------------------+-----------------+----------------+
|181|Sleeping With Sirens|If I'm James Dean...|With Ears To See ...|              66|['neon pop punk',...|               62|         2241737|
|526|                FLOW|                Sign|           MICROCOSM|              68|           ['anime']|               57|          455986|
|594|             Wig Wam|Do Ya Wanna Taste It|Non Stop Rock'n Roll|              61|       ['glam rock']|               43|           36853|
|395|Ramon Ayala Y Sus...|      Tragos Amargos|  Cruzando Fronteras|              56|['musica mexicana...|               62|    

In [500]:
#after processing data in PySpark convert to Pandas DataFrame:
df = merged_df.toPandas()
df.head()

,_c0,artist,track,album,track_popularity,artist_genres,artist_popularity,artist_followers
0,181,Sleeping With Sirens,"If I'm James Dean, You're Audrey Hepburn",With Ears To See And Eyes To Hear,66,"['neon pop punk', 'pop emo', 'pop punk', 'scre...",62,2241737
1,526,FLOW,Sign,MICROCOSM,68,['anime'],57,455986
2,594,Wig Wam,Do Ya Wanna Taste It,Non Stop Rock'n Roll,61,['glam rock'],43,36853
3,395,Ramon Ayala Y Sus Bravos Del Norte,Tragos Amargos,Cruzando Fronteras,56,"['musica mexicana', 'norteno']",62,468728
4,146,Jason Derulo,In My Head,Jason Derulo,64,"['dance pop', 'pop']",77,11929946


In [501]:
#drop unecessary column:
df = df.drop(["_c0"], axis=1)

In [502]:
df

,artist,track,album,track_popularity,artist_genres,artist_popularity,artist_followers
0,Sleeping With Sirens,"If I'm James Dean, You're Audrey Hepburn",With Ears To See And Eyes To Hear,66,"['neon pop punk', 'pop emo', 'pop punk', 'scre...",62,2241737
1,FLOW,Sign,MICROCOSM,68,['anime'],57,455986
2,Wig Wam,Do Ya Wanna Taste It,Non Stop Rock'n Roll,61,['glam rock'],43,36853
3,Ramon Ayala Y Sus Bravos Del Norte,Tragos Amargos,Cruzando Fronteras,56,"['musica mexicana', 'norteno']",62,468728
4,Jason Derulo,In My Head,Jason Derulo,64,"['dance pop', 'pop']",77,11929946
...,...,...,...,...,...,...,...
253,Pierce The Veil,Million Dollar Houses (The Painter),Selfish Machines,58,"['pop punk', 'screamo']",71,2616964
254,Waka Flocka Flame,Grove St. Party (feat. Kebo Gotti),Flockaveli,64,"['atl hip hop', 'dirty south rap', 'pop rap', ...",64,2714590
255,Avenged Sevenfold,Fiction,Nightmare,54,"['alternative metal', 'nu metal']",74,6508554
256,Enrique Iglesias,Cuando Me Enamoro,Euphoria (Standard US/Latin version),74,"['dance pop', 'latin pop', 'mexican pop']",75,11327933


In [368]:
spark.stop()

#### 4. Exploratory Data Analysis(EDA):
  

In [563]:
#pandas dataframe created from pyspark df:
df

,artist,track,album,track_popularity,artist_genres,artist_popularity,artist_followers
0,Sleeping With Sirens,"If I'm James Dean, You're Audrey Hepburn",With Ears To See And Eyes To Hear,66,"[hip hop, pop rap, gospelnhip hop, pop rap, go...",62,2241737
1,FLOW,Sign,MICROCOSM,68,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",57,455986
2,Wig Wam,Do Ya Wanna Taste It,Non Stop Rock'n Roll,61,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",43,36853
3,Ramon Ayala Y Sus Bravos Del Norte,Tragos Amargos,Cruzando Fronteras,56,"[hip hop, pop rap, gospelmuhip hop, pop rap, g...",62,468728
4,Jason Derulo,In My Head,Jason Derulo,64,"[hip hop, pop rap, gospeldhip hop, pop rap, go...",77,11929946
...,...,...,...,...,...,...,...
253,Pierce The Veil,Million Dollar Houses (The Painter),Selfish Machines,58,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",71,2616964
254,Waka Flocka Flame,Grove St. Party (feat. Kebo Gotti),Flockaveli,64,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",64,2714590
255,Avenged Sevenfold,Fiction,Nightmare,54,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",74,6508554
256,Enrique Iglesias,Cuando Me Enamoro,Euphoria (Standard US/Latin version),74,"[hip hop, pop rap, gospeldhip hop, pop rap, go...",75,11327933


In [504]:
#show basic data info:
df.info()
#data shape:
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             258 non-null    object
 1   track              258 non-null    object
 2   album              258 non-null    object
 3   track_popularity   258 non-null    int32 
 4   artist_genres      258 non-null    object
 5   artist_popularity  258 non-null    object
 6   artist_followers   258 non-null    object
dtypes: int32(1), object(6)
memory usage: 13.2+ KB


(258, 7)

In [ ]:
df

In [505]:
#Find the duplicates
df.duplicated().sum()

0

In [506]:
#describe the data:--->to be added audio features when finally loaded
df.describe()

,track_popularity
count,258.000000
mean,65.968992
std,8.479720
min,49.000000
25%,59.000000
50%,66.000000
75%,72.000000
max,89.000000


In [507]:
#check if there is any correlation between track popuarity and audio features of the track:
scatter = sns.pairplot(df[["track_popularity", "acousticness", "danceability","energy", "instrumentalness","loudness","tempo"]], diag_kind="kde")

In [525]:
#heatmap to summarize the correlation ratio:
numeric=["track_popularity", "acousticness", "danceability", "energy", "instrumentalness", "loudness", "tempo"]
correlation_matrix = df[numeric].corr()
y, x = plt.subplots(figsize=(9, 7))

heatmap = sns.heatmap(matrix, annot=True, fmt=".2f", linewidths=.5, cmap="GnBu")
plt.show()

KeyError: "['acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'tempo'] not in index"

In [572]:
#track audio_feature pairplot:
sns.pairplot(df)
plt.show()

In [573]:
#fit a linear regression model to the relationship between track audio feature - danceability and track popularity. 
model = LinearRegression()

x = df[["danceability"]]
y = df["track_popularity"]
model.fit(x, y)

predictions = model.predict(X)

plt.scatter(x, y, label="Actual", alpha=0.5)
plt.plot(x, predictions, color="green", label="Predicted")
plt.xlabel("danceability")
plt.ylabel("track_popularity")
plt.title("Linear Regression: danceability vs. track popularity")
plt.legend()
plt.show()

KeyError: "None of [Index(['danceability'], dtype='object')] are in the [columns]"

#### 5. Graph Analysis:

#### 6. Data Visualization:

In [593]:
#top 10 most popular tracks of 2010 on Spotify:
top_10 = df.sort_values("track_popularity", ascending=False)
top_10.head(10)

,artist,track,album,track_popularity,artist_genres,artist_popularity,artist_followers
251,Miguel,Sure Thing,All I Want Is You,89,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",77,4788498
58,Bruno Mars,Just the Way You Are,Doo-Wops & Hooligans,86,"[hip hop, pop rap, gospeldhip hop, pop rap, go...",87,50939168
214,Patrick Watson,Je te laisserai des mots,Je te laisserai des mots,85,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",68,610904
93,Train,"Hey, Soul Sister","Save Me, San Francisco (Golden Gate Edition)",85,"[hip hop, pop rap, gospeldhip hop, pop rap, go...",71,4356940
185,Mac Miller,The Spins,K.I.D.S. (Deluxe),84,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",82,9497713
237,Rihanna,Only Girl (In The World),Loud (Japan Version),84,"[hip hop, pop rap, gospelbhip hop, pop rap, go...",88,57812873
150,Justin Bieber,Baby,My World 2.0,83,"[hip hop, pop rap, gospelchip hop, pop rap, go...",87,74210660
72,Roar,Christmas Kids,I Can't Handle Change,82,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",66,706938
116,Nicki Minaj,Super Bass,Pink Friday (Complete Edition),82,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",86,29523913
84,Miguel,Girl With The Tattoo Enter.lewd,All I Want Is You,82,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",77,4788498


In [621]:
#plot top 10 tracks on Spotify 2010:
fig=px.line(top_10.head(10),x="track",y="track_popularity",hover_data=["artist"],color_discrete_sequence=["green"],markers=True,title="Top 10 tracks on Spotify 2010")
fig.show()

In [663]:
#top 20 most popular artists of 2010 on Spotify:
top_artists = df.sort_values("artist_popularity", ascending=False)
top_artists

,artist,track,album,track_popularity,artist_genres,artist_popularity,artist_followers
94,Drake,Up All Night,Thank Me Later,68,"[hip hop, pop rap, gospelchip hop, pop rap, go...",94,79671951
19,Drake,Find Your Love,Thank Me Later,70,"[hip hop, pop rap, gospelchip hop, pop rap, go...",94,79671951
67,Drake,The Resistance,Thank Me Later,63,"[hip hop, pop rap, gospelchip hop, pop rap, go...",94,79671951
171,Drake,Over,Thank Me Later,73,"[hip hop, pop rap, gospelchip hop, pop rap, go...",94,79671951
36,Kanye West,POWER,My Beautiful Dark Twisted Fantasy,80,"[hip hop, pop rap, gospelchip hop, pop rap, go...",89,21267742
...,...,...,...,...,...,...,...
180,Orianthi,According To You,Believe (II),50,"[hip hop, pop rap, gospelchip hop, pop rap, go...",40,82596
115,Garrett Hedlund,Give In to Me,Country Strong (More Music from the Motion Pic...,55,[],38,20459
8,Dondria,You're the One,Dondria vs. Phatfffat,51,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",33,220069
202,Taylor Swift,Never Grow Up,Speak Now (Deluxe Edition),55,"[hip hop, pop rap, gospelhip hop, pop rap, gos...",100,86026925


In [666]:
fig = px.bar(top_artists.iloc[:20,:].sort_values(by=["artist_popularity"], ascending=False), 
             x="artist", y="artist_popularity")
fig.update_layout(
    title={"text": "Top Artists on Spotify 2010" })
fig.show()

In [665]:
#remove specific  characters from df column "artist_genres": ---> to improve the plot for genres
#df["artist_genres"] = df["artist_genres"].str.replace(, regex=True)

In [629]:
#check popular genres based on popularity in 2010:
fig=px.histogram(df["artist_genres"].sort_values("track_popularity",ascending=False),x="artist_genres",y="track_popularity",color_discrete_sequence=["yellow"],marginal='box',title="Popular genres based on track pouplarity 2010")
fig.update_layout(title_x=0.5)

TypeError: Series.sort_values() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given

#### 7. Insights and Reporting:

#### 8. Performance Optimization: